In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
tour = validation_data_input.get_data('tour')

In [3]:
df_tour = tour.copy()
mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: walk to transit",
            7: "7: park-and-ride",
            8: "8: school bus",
            9: "9:tnc"}
pdpurp_cat = {1: "1: Work",
              2: "2: School",
              3: "3: Escort",
              4: "4: other home-based",
              5: "4: other home-based",
              6: "4: other home-based",
              7: "4: other home-based",
              8: "4: other home-based",
              9: "4: other home-based",
              10: "4: other home-based"}

df_tour['tmodetp_label'] = df_tour['tmodetp'].map(mode_cat)
df_tour['pdpurp_label'] = df_tour['pdpurp'].map(pdpurp_cat)

In [4]:
def plot_mode_choice(df:pd.DataFrame, title_name:str):
    df_plot = df.groupby(['source','pdpurp_label','tmodetp_label'])['toexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source','pdpurp_label'], group_keys=False)['toexpfac']. \
        apply(lambda x: x / float(x.sum()))
    
    df_plot_ct = df.groupby(['source','pdpurp_label','tmodetp_label'])['toexpfac'].count().reset_index(). \
        rename(columns={'toexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source','pdpurp_label','tmodetp_label'])
    
    fig = px.bar(df_plot.sort_values(by=['pdpurp_label','source']), x="tmodetp_label", y="percentage", color="source",
                 facet_col='pdpurp_label', facet_col_wrap=2,
                 barmode="group",hover_data=['sample count'],title=title_name)
    fig.update_layout(height=600, width=900, font=dict(size=11),
                      xaxis = dict(dtick = 1, categoryorder='category ascending'))
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
    fig.show()

In [5]:
all_tour = df_tour.loc[df_tour['parent']==0].copy()
plot_mode_choice(all_tour,"tour mode choice by purpose")